# Day 10 - Decorators and Context Managers in Python

## Why Are Decorators and Context Managers Important?
Decorators are powerful tools in Python that allow you to extend or enhance the behavior of existing functions or methods in a clean and reusable manner. They are especially useful for adding functionality like performance monitoring, access control, and logging, without modifying the original function's body. This leads to more maintainable and understandable code.

Context managers, on the other hand, are essential for proper resource management. They provide a structured way to allocate and release resources—whether it's file streams, database connections, or network connections—when needed. By ensuring resources are released after use, context managers help prevent resource leaks and make your code safer and cleaner.

## Understanding Decorators and Context Managers
Decorators allow you to wrap a function with additional functionality. They are commonly used for logging, access control, memoization, and more.
A decorator is applied using the `@decorator_name` syntax above the function definition.

In [2]:
# Define the decorator function
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


### Explanation:
- **`my_decorator(func)`**: This is the decorator function. It takes a single argument, which is another function (`func`).
- **`wrapper()`**: Inside `my_decorator`, we define another function called `wrapper()`. This function wraps the execution of the original function `func()`. It will be responsible for adding extra functionality.
- **`return wrapper`**: The decorator function (`my_decorator`) does not return the original function itself but returns the `wrapper` function.

## Context Managers
A context manager is an object that manages resources (like files or connections) during a code block.  
The most common use case is with the `with` statement, which ensures resources are properly allocated and released.

In [3]:
class ManagedFile:
    def __init__(self, filename):
        # Initialize with the name of the file
        self.filename = filename

    def __enter__(self):
        # Open the file and return the file object to be used in the 'with' block
        self.file = open(self.filename, 'w')
        print(f"Opening file {self.filename}")
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Ensure the file is closed no matter what happens in the 'with' block
        if self.file:
            self.file.close()
            print(f"Closing file {self.filename}")
        # Handle any exceptions (if necessary)
        if exc_type is not None:
            print(f"Exception occurred: {exc_type}, {exc_val}")
        return True

with ManagedFile('example.txt') as f:
    f.write('Hello, world!')
    raise Exception("An error occurred while writing to the file")

Opening file example.txt
Closing file example.txt
Exception occurred: <class 'Exception'>, An error occurred while writing to the file


### Explanation of the Code:
- **`__enter__()`**: This method is called at the start of the `with` block. It opens the file in write mode and returns the file object to be used.
- **`__exit__()`**: This method is called after the code block is done, even if an exception occurs. It closes the file and prints a message.

## Step-by-Step Explanation: Creating a Logging Decorator
The goal of the logging decorator is to measure and log the execution time of any function it wraps.

In [4]:
import time

def log_execution_time(func):
    def wrapper(*args, **kwargs):
        # Record the start time
        start_time = time.time()

        # Execute the original function
        result = func(*args, **kwargs)

        # Record the end time and calculate the duration
        end_time = time.time()
        execution_time = end_time - start_time

        # Log the execution time
        print(f"Executed {func.__name__} in {execution_time:.4f} seconds")

        return result
    return wrapper

@log_execution_time
def slow_function(seconds):
    """Simulates a slow function by sleeping for a given number of seconds."""
    time.sleep(seconds)
    return f"Slept for {seconds} seconds"

print(slow_function(2))

Executed slow_function in 2.0009 seconds
Slept for 2 seconds


## Adding a Context Manager for Logging to a File:
This context manager ensures the log file is opened before writing and is properly closed afterward.

In [5]:
class LogFileManager:
    def __init__(self, log_file):
        self.log_file = log_file

    def __enter__(self):
        self.file = open(self.log_file, 'a')  # Open the log file in append mode
        return self.file

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if self.file:
            self.file.close()  # Ensure the file is closed

In [6]:
def log_execution_time_to_file(log_file):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = end_time - start_time

            # Log execution time to both console and log file
            print(f"Executed {func.__name__} in {execution_time:.4f} seconds")
            
            # Using context manager to write to log file
            with LogFileManager(log_file) as log:
                log.write(f"Executed {func.__name__} in {execution_time:.4f} seconds\n")

            return result
        return wrapper
    return decorator

@log_execution_time_to_file('execution_times.log')
def slow_function(seconds):
    """Simulates a slow function by sleeping for a given number of seconds."""
    time.sleep(seconds)
    return f"Slept for {seconds} seconds"

# Run the function to test logging
print(slow_function(2))

Executed slow_function in 2.0010 seconds
Slept for 2 seconds
